In [1]:
%load_ext autoreload
%autoreload 2

# Super Weights Analysis - Identifying Super Weights and their Impact on Model Performance

In [2]:
from research.researcher import SuperWeightResearchSession
import torch

In [3]:
# model_name = 'mistralai/Mistral-7B-v0.1'
# model_name = 'meta-llama/Llama-2-7b-hf'
model_name = 'OLMo-1B-0724-hf'

In [4]:
session = SuperWeightResearchSession.from_model_name(model_name, cache_dir='~/models/')

Found locally downloaded model at /Users/fabiangrob/models/OLMo-1B-0724-hf


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

2025-09-01 02:22:14,169 - SuperWeightManager_13335784384 - INFO - SuperWeightManager initialized with shared MLP handler
2025-09-01 02:22:14,172 - SuperWeightDetector_13335784480 - INFO - SuperWeightDetector initialized for -Users-fabiangrob-models-OLMo-1B-0724-hf
2025-09-01 02:22:14,172 - SuperWeightDetector_13335784480 - INFO - Model has 16 layers
2025-09-01 02:22:14,172 - SuperWeightDetector_13335784480 - INFO - SuperWeightDetector initialized with manager integration
2025-09-01 02:22:14,173 - SuperWeightResearch_13275716896 - INFO - SuperWeightResearchSession initialized
2025-09-01 02:22:14,173 - SuperWeightResearch_13275716896 - INFO - Model: -Users-fabiangrob-models-OLMo-1B-0724-hf
2025-09-01 02:22:14,174 - SuperWeightResearch_13275716896 - INFO - Architecture: gated_mlp
2025-09-01 02:22:14,174 - SuperWeightResearch_13275716896 - INFO - Using detector: SuperWeightDetector


In [5]:
session.model

OlmoForCausalLM(
  (model): OlmoModel(
    (embed_tokens): Embedding(50304, 2048, padding_idx=1)
    (layers): ModuleList(
      (0-15): 16 x OlmoDecoderLayer(
        (self_attn): OlmoAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (v_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): OlmoMLP(
          (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): OlmoLayerNorm()
        (post_attention_layernorm): OlmoLayerNorm()
      )
    )
    (norm): OlmoLayerNorm()
    (rotary_emb): OlmoRotaryEmbedding()
  )
  (lm_head): 

In [6]:
sw = session.detect_super_weights(spike_threshold=70.0)

2025-09-01 02:22:15,457 - SuperWeightResearch_13275716896 - INFO - Starting super weight detection
2025-09-01 02:22:15,458 - SuperWeightDetector_13335784480 - INFO - Starting super weight detection
2025-09-01 02:22:15,458 - SuperWeightDetector_13335784480 - INFO - Parameters: threshold=70.0, max_iterations=10
2025-09-01 02:22:15,458 - SuperWeightDetector_13335784480 - INFO - Zero detected weights: True
2025-09-01 02:22:15,462 - SuperWeightDetector_13335784480 - INFO - === Iteration 1 ===
2025-09-01 02:22:15,879 - SuperWeightDetector_13335784480 - INFO - Found 2 potential super weights in iteration 1
2025-09-01 02:22:15,879 - SuperWeightDetector_13335784480 - INFO - Zeroing 2 detected super weights...
2025-09-01 02:22:15,909 - SuperWeightManager_13335784384 - INFO - Successfully scaled 2/2 super weights by 0.000
2025-09-01 02:22:15,909 - SuperWeightDetector_13335784480 - INFO - Found 2 new super weights:
2025-09-01 02:22:15,909 - SuperWeightDetector_13335784480 - INFO -   1. Layer 1 mlp

In [7]:
sw

[SuperWeight(layer=1, coords=[1764, 1710], input=-401.50, output=-262.00),
 SuperWeight(layer=15, coords=[1764, 6840], input=-283.00, output=415.00),
 SuperWeight(layer=2, coords=[1764, 8041], input=345.25, output=-227.50),
 SuperWeight(layer=15, coords=[1764, 3454], input=-170.38, output=145.88)]

## Vocabulary analysis

In [ ]:
# Enhanced analysis with Universal Neurons processing
enhanced_analysis = session.analyzer.vocabulary_analyzer.analyze_neuron_vocabulary_effects(
    sw[0], 
    apply_universal_neurons_processing=True  # Enable layer norm folding & mean-centering
)

# Display the enhanced results
session.analyzer.vocabulary_analyzer.display_vocabulary_card(enhanced_analysis, 10)

=== Vocabulary Effect Card: Layer 1 mlp.down_proj.weight[1764, 1710] ===
Processing applied: True

--- Raw Dot Product Analysis ---
Classification: UNCLEAR (0.15)
  Effects too small or distributed to classify clearly
Moments: var=0.000, skew=0.20, kurt=-3.00
Percentiles: p5=-0.010, p50=-0.000, p95=0.012
Best theme: whitespace_prefixed (score: 0.126)
Top 10 boosted:
  1. ' �' (+0.038)
  2. ' vet' (+0.031)
  3. ' stir' (+0.031)
  4. ' appeal' (+0.031)
  5. ' appro' (+0.031)
  6. ' comm' (+0.031)
  7. ' Polish' (+0.030)
  8. ' plac' (+0.030)
  9. ' bro' (+0.029)
  10. ' MOT' (+0.029)
Top 10 suppressed:
  1. ' mar' (-0.034)
  2. ' mutual' (-0.031)
  3. ' Bell' (-0.029)
  4. '039' (-0.028)
  5. ' corresponding' (-0.028)
  6. ' RE' (-0.027)
  7. ' enormous' (-0.027)
  8. ' notebook' (-0.027)
  9. ' Final' (-0.027)
  10. ' respective' (-0.027)

--- Cosine Similarity Analysis ---
Classification: UNCLEAR (0.01)
Moments: var=0.000, skew=-0.13, kurt=-0.24
Percentiles: p5=-0.021, p50=-0.001, p95=

In [ ]:
# Analyze intervention effects with robust windowed evaluation
intervention_results = session.analyzer.vocabulary_analyzer.analyze_super_weight_intervention(
    sw[0],
    n_samples=100  # Uses token-level filtering now
)

# Display intervention results
session.analyzer.vocabulary_analyzer.display_intervention_results(intervention_results)

2025-09-01 02:27:31,848 - SuperWeightManager_13335784384 - INFO - Successfully scaled 1/1 super weights by 0.000
2025-09-01 02:27:45,586 - SuperWeightManager_13335784384 - INFO - Restored 1/1 weights


=== Intervention Effect Card: Layer 1 mlp.down_proj.weight[1764, 1710] ===
Loss: 3.3462 → 8.6478 (Δ+5.3016)
Entropy: 3.2634 → 5.2730 (Δ+2.0096)
Top-K Margin: 0.3086 → 0.2235 (Δ-0.0851)
Stopword Mass: 0.1672 → 0.0607 (Δ-0.1066)
Evaluated on 100 samples


In [ ]:
# Run comprehensive control experiments
control_results = session.analyzer.vocabulary_analyzer.analyze_controls_and_baselines(
    sw[0],
    n_samples=100
)

print("=== Control Experiments ===")
print(f"Super Weight vs Random Baseline:")
print(f"  SW delta loss: {control_results['random_baseline']['sw_delta_loss']:+.4f}")
print(f"  Random delta loss: {control_results['random_baseline']['random_delta_loss']:+.4f}")
print(f"  Specificity ratio: {control_results['random_baseline']['specificity_ratio']:.2f}x")

print(f"\nNeuron vs Scalar:")
print(f"  Scalar contribution: {control_results['neuron_vs_scalar']['scalar_contribution_ratio']:.2f}")

print(f"\nDeterministic check:")
print(f"  Is deterministic: {control_results['noop_check']['is_deterministic']}")

2025-09-01 02:28:11,527 - SuperWeightManager_13335784384 - INFO - Successfully scaled 1/1 super weights by 0.000
2025-09-01 02:28:24,102 - SuperWeightManager_13335784384 - INFO - Restored 1/1 weights
2025-09-01 02:28:36,538 - SuperWeightManager_13335784384 - INFO - Successfully scaled 1/1 super weights by 0.000
2025-09-01 02:28:48,911 - SuperWeightManager_13335784384 - INFO - Restored 1/1 weights
2025-09-01 02:29:01,348 - SuperWeightManager_13335784384 - INFO - Successfully scaled 1/1 super weights by 0.000
2025-09-01 02:29:13,587 - SuperWeightManager_13335784384 - INFO - Restored 1/1 weights


=== Control Experiments ===
Super Weight vs Random Baseline:
  SW delta loss: +5.2069
  Random delta loss: +5.2069
  Specificity ratio: 1.00x

Neuron vs Scalar:
  Scalar contribution: 1.38

Deterministic check:
  Is deterministic: True


In [ ]:
# Analyze cascade effects with custom parameters
cascade_results = session.analyzer.vocabulary_analyzer.analyze_cascade_effects(
    sw[0],
    input_text="Apple Inc. develops innovative technology solutions.",
    num_layers=8,  # Sample 8 layers instead of default 5
    top_k_margin=15  # Use top-15 for margin calculation instead of default 10
)

# Display with the new method that shows actual layer indices
session.analyzer.vocabulary_analyzer.display_cascade_results(cascade_results)

# Now you can see which exact layers were analyzed!
print(f"Actual layers analyzed: {cascade_results['actual_layer_indices']}")
# Example output: [0, 4, 8, 12, 16, 20, 24]

2025-09-01 03:08:48,953 - SuperWeightManager_13335784384 - INFO - Successfully scaled 1/1 super weights by 0.000
2025-09-01 03:08:49,051 - SuperWeightManager_13335784384 - INFO - Restored 1/1 weights


=== Cascade Effect Card: Layer 1 mlp.down_proj.weight[1764, 1710] ===
Input: Apple Inc. develops innovative technology solutions.
Pattern: accelerating
Layers analyzed: 8
Actual layer indices: [0, 2, 4, 6, 8, 10, 12, 14]
Top-K margin parameter: 15

Layer-by-layer effects:
  Layer 0: entropy_Δ=+0.0000, margin_Δ=+0.0000, magnitude=0.0000
  Layer 2: entropy_Δ=+0.0078, margin_Δ=-0.0242, magnitude=15.3984
  Layer 4: entropy_Δ=+0.0000, margin_Δ=-0.0254, magnitude=14.8125
  Layer 6: entropy_Δ=+0.0000, margin_Δ=+0.0061, magnitude=16.8594
  Layer 8: entropy_Δ=+0.0078, margin_Δ=-0.0100, magnitude=18.9219
  Layer 10: entropy_Δ=-0.0078, margin_Δ=+0.0154, magnitude=21.1094
  Layer 12: entropy_Δ=-0.0078, margin_Δ=-0.0742, magnitude=26.7031
  Layer 14: entropy_Δ=-0.0312, margin_Δ=-0.1768, magnitude=44.9062

Actual layers analyzed: [0, 2, 4, 6, 8, 10, 12, 14]


In [ ]:
# Compare enhanced vs standard analysis
print("=== Enhanced vs Standard Analysis Comparison ===")

# Enhanced (with Universal Neurons processing)
enhanced = session.analyzer.vocabulary_analyzer.analyze_neuron_vocabulary_effects(
    sw[0], 
    apply_universal_neurons_processing=True
)

# Standard (without processing)
standard = session.analyzer.vocabulary_analyzer.analyze_neuron_vocabulary_effects(
    sw[0], 
    apply_universal_neurons_processing=False
)

print("Enhanced Analysis:")
session.analyzer.vocabulary_analyzer.display_vocabulary_card(enhanced)

print("\n" + "="*50)
print("Standard Analysis:")
session.analyzer.vocabulary_analyzer.display_vocabulary_card(standard)

=== Enhanced vs Standard Analysis Comparison ===
Enhanced Analysis:
=== Vocabulary Effect Card: Layer 1 mlp.down_proj.weight[1764, 1710] ===
Processing applied: True

--- Raw Dot Product Analysis ---
Classification: UNCLEAR (0.15)
  Effects too small or distributed to classify clearly
Moments: var=0.000, skew=0.20, kurt=-3.00
Percentiles: p5=-0.010, p50=-0.000, p95=0.012
Best theme: whitespace_prefixed (score: 0.126)
Top 5 boosted:
  1. ' �' (+0.038)
  2. ' vet' (+0.031)
  3. ' stir' (+0.031)
  4. ' appeal' (+0.031)
  5. ' appro' (+0.031)
Top 5 suppressed:
  1. ' mar' (-0.034)
  2. ' mutual' (-0.031)
  3. ' Bell' (-0.029)
  4. '039' (-0.028)
  5. ' corresponding' (-0.028)

--- Cosine Similarity Analysis ---
Classification: UNCLEAR (0.01)
Moments: var=0.000, skew=-0.13, kurt=-0.24
Percentiles: p5=-0.021, p50=-0.001, p95=0.019
Classification: UNCLEAR (0.15)
  Effects too small or distributed to classify clearly
Moments: var=0.00, skew=0.20, kurt=-3.00
Best theme: whitespace_prefixed (sco

In [ ]:
# Analyze enrichment in specific token classes
analysis = session.analyzer.vocabulary_analyzer.analyze_neuron_vocabulary_effects(
    sw[0], apply_universal_neurons_processing=True
)

enrichment = analysis['raw_analysis']['enrichment']
print("=== Token Class Enrichment ===")
print(f"Best enriched class: {enrichment['best_theme']['class']}")
print(f"Enrichment score: {enrichment['best_theme']['score']:.3f}")
print(f"Description: {enrichment['best_theme']['description']}")

print("\nAll enrichment scores:")
for class_name, score in enrichment['enrichment_scores'].items():
    if score > 0.1:  # Only show significant enrichments
        effects = enrichment['class_effects'][class_name]
        print(f"  {class_name}: {score:.3f} (mean={effects['mean']:+.3f}, count={effects['count']})")

=== Token Class Enrichment ===
Best enriched class: whitespace_prefixed
Enrichment score: 0.126
Description: Effects concentrated in whitespace_prefixed tokens

All enrichment scores:
  whitespace_prefixed: 0.126 (mean=+0.002, count=28854)


## Perplexity and Accuracy analysis

In [13]:
sw[0]

SuperWeight(layer=1, coords=[2070, 7310], input=950.00, output=-262.75)

In [14]:
perplexity_impact = session.analyzer.metrics_analyzer.measure_perplexity_impact(sw[0], n_samples=100)

2025-07-04 16:12:08,000 - SuperWeightManager_140164721294544 - INFO - Successfully scaled 1/1 super weights by 0.000
2025-07-04 16:12:16,832 - SuperWeightManager_140164721294544 - INFO - Restored 1/1 weights


In [15]:
perplexity_impact

{'baseline_perplexity': 10.63492488861084,
 'modified_perplexity': 5289.5185546875,
 'perplexity_ratio': 497.3724412809116,
 'perplexity_increase': 5278.883629798889,
 'impact_severity': 'catastrophic',
 'dataset_info': {'name': 'wikitext',
  'config': 'wikitext-2-raw-v1',
  'split': 'test',
  'n_samples': 100},
 'super_weight': SuperWeight(layer=1, coords=[2070, 7310], input=950.00, output=-262.75)}

In [32]:
for super_weight in sw:
    print(super_weight)
    ppl_res = session.analyzer.metrics_analyzer.measure_perplexity_impact(super_weight, n_samples=500)
    print(f"Perplexity impact for {super_weight}: {ppl_res}")

Layer 1 mlp.down_proj.weight[788, 2427]


2025-07-04 13:58:02,444 - SuperWeightManager_139901926293312 - INFO - Successfully scaled 1/1 super weights by 0.000
2025-07-04 13:58:49,356 - SuperWeightManager_139901926293312 - INFO - Restored 1/1 weights


Perplexity impact for Layer 1 mlp.down_proj.weight[788, 2427]: {'baseline_perplexity': 11.002143859863281, 'modified_perplexity': 11.295992851257324, 'perplexity_ratio': 1.0267083393143066, 'perplexity_increase': 0.29384899139404297, 'impact_severity': 'minimal', 'dataset_info': {'name': 'wikitext', 'config': 'wikitext-2-raw-v1', 'split': 'test', 'n_samples': 500}, 'super_weight': SuperWeight(layer=1, coords=[788, 2427], input=-443.50, output=300.50)}
Layer 31 mlp.down_proj.weight[788, 12111]


2025-07-04 13:59:38,902 - SuperWeightManager_139901926293312 - INFO - Successfully scaled 1/1 super weights by 0.000
2025-07-04 14:00:25,809 - SuperWeightManager_139901926293312 - INFO - Restored 1/1 weights


Perplexity impact for Layer 31 mlp.down_proj.weight[788, 12111]: {'baseline_perplexity': 11.002143859863281, 'modified_perplexity': 11.023534774780273, 'perplexity_ratio': 1.0019442497016449, 'perplexity_increase': 0.021390914916992188, 'impact_severity': 'minimal', 'dataset_info': {'name': 'wikitext', 'config': 'wikitext-2-raw-v1', 'split': 'test', 'n_samples': 500}, 'super_weight': SuperWeight(layer=31, coords=[788, 12111], input=151.00, output=-299.75)}


In [51]:
# check impact of all super weights at once
all_sw_ppl = session.analyzer.metrics_analyzer.measure_perplexity_impact(sw, n_samples=500)
print(f"Perplexity impact for all super weights: {all_sw_ppl}")

2025-07-04 14:16:07,042 - SuperWeightManager_139901044718128 - INFO - Successfully scaled 2/2 super weights by 0.000
2025-07-04 14:16:51,120 - SuperWeightManager_139901044718128 - INFO - Restored 2/2 weights


Perplexity impact for all super weights: {'baseline_perplexity': 9.51281452178955, 'modified_perplexity': 4830.10791015625, 'perplexity_ratio': 507.7475124835725, 'perplexity_increase': 4820.59509563446, 'impact_severity': 'catastrophic', 'dataset_info': {'name': 'wikitext', 'config': 'wikitext-2-raw-v1', 'split': 'test', 'n_samples': 500}, 'super_weights': [SuperWeight(layer=1, coords=[2070, 7310], input=950.00, output=-262.75), SuperWeight(layer=31, coords=[53, 2187], input=-906.00, output=-84.50)], 'num_weights': 2, 'average_impact_per_weight': 2410.29754781723}


In [52]:
accuracy_impact = session.analyzer.metrics_analyzer.measure_accuracy_impact(sw[0], n_samples=100)

2025-07-04 14:18:35,674 - SuperWeightManager_139901044718128 - INFO - Successfully scaled 1/1 super weights by 0.000
2025-07-04 14:19:04,362 - SuperWeightManager_139901044718128 - INFO - Restored 1/1 weights


In [53]:
accuracy_impact

{'task': 'hellaswag',
 'baseline_accuracy': 0.64,
 'modified_accuracy': 0.33,
 'accuracy_drop': 0.31,
 'accuracy_ratio': 0.515625,
 'impact_severity': 'severe',
 'n_samples': 100,
 'super_weight': SuperWeight(layer=1, coords=[2070, 7310], input=950.00, output=-262.75)}

In [19]:
for super_weight in sw:
    print(super_weight)
    acc_res = session.analyzer.metrics_analyzer.measure_accuracy_impact(super_weight, n_samples=200)
    print(f"Accuracy impact for {super_weight}: {acc_res}")

Layer 1 mlp.down_proj.weight[1764, 1710]


2025-06-10 01:51:27,451 - SuperWeightManager_13442570496 - INFO - Successfully scaled 1/1 super weights by 0.000
2025-06-10 01:52:12,462 - SuperWeightManager_13442570496 - INFO - Restored 1/1 weights


Accuracy impact for Layer 1 mlp.down_proj.weight[1764, 1710]: {'super_weight': SuperWeight(layer=1, coords=[1764, 1710], input=-401.50, output=-262.00), 'task': 'hellaswag', 'baseline_accuracy': 0.475, 'modified_accuracy': 0.355, 'accuracy_drop': 0.12, 'accuracy_ratio': 0.7473684210526316, 'impact_severity': 'moderate', 'n_samples': 200}
Layer 3 mlp.down_proj.weight[1764, 1902]


2025-06-10 01:52:59,266 - SuperWeightManager_13442570496 - INFO - Successfully scaled 1/1 super weights by 0.000
2025-06-10 01:53:44,296 - SuperWeightManager_13442570496 - INFO - Restored 1/1 weights


Accuracy impact for Layer 3 mlp.down_proj.weight[1764, 1902]: {'super_weight': SuperWeight(layer=3, coords=[1764, 1902], input=-63.56, output=-64.69), 'task': 'hellaswag', 'baseline_accuracy': 0.475, 'modified_accuracy': 0.465, 'accuracy_drop': 0.009999999999999953, 'accuracy_ratio': 0.9789473684210527, 'impact_severity': 'minimal', 'n_samples': 200}
Layer 15 mlp.down_proj.weight[1764, 6840]


2025-06-10 01:54:30,827 - SuperWeightManager_13442570496 - INFO - Successfully scaled 1/1 super weights by 0.000
2025-06-10 01:55:15,710 - SuperWeightManager_13442570496 - INFO - Restored 1/1 weights


Accuracy impact for Layer 15 mlp.down_proj.weight[1764, 6840]: {'super_weight': SuperWeight(layer=15, coords=[1764, 6840], input=-283.00, output=415.00), 'task': 'hellaswag', 'baseline_accuracy': 0.475, 'modified_accuracy': 0.47, 'accuracy_drop': 0.0050000000000000044, 'accuracy_ratio': 0.9894736842105263, 'impact_severity': 'minimal', 'n_samples': 200}


## Detailed mathematical break down of super activation

In [29]:
mathematical_analysis = session.analyzer.mathematical_super_activation_analysis(sw[0])

2025-09-01 03:12:11,893 - SuperWeightAnalyzer_13066373392 - INFO - Mathematical super activation analysis for Layer 1 mlp.down_proj.weight[1764, 1710]
2025-09-01 03:12:11,895 - SuperActivationAnalyzer_13335784816 - INFO - Super activation analysis for Layer 1 mlp.down_proj.weight[1764, 1710]
2025-09-01 03:12:11,896 - SuperActivationAnalyzer_13335784816 - INFO - Detected architecture: MLPArchitectureType.GATED_MLP


In [30]:
mathematical_analysis['mathematical_breakdown']

{'input_analysis': {'input_vector_shape': [2048],
  'input_magnitude': 45.21875,
  'input_mean': 4.470348358154297e-06,
  'input_std': 0.99951171875},
 'step1_gate_projection': {'operation': 'gate_output = W_gate @ x',
  'bias_term': None,
  'result_target_channel': 15.2421875,
  'has_bias': False},
 'step2_activation': {'operation': 'activated_gate = SILU(gate_output)',
  'input_to_activation': 15.2421875,
  'activation_output_target_channel': 15.2421875,
  'activation_function': 'silu',
  'activation_derivative_at_point': 1.000003395600431},
 'step3_up_projection': {'operation': 'up_output = W_up @ x',
  'bias_term': None,
  'result_target_channel': -26.34375,
  'has_bias': False},
 'step4_hadamard_product': {'operation': 'hadamard_result = activated_gate ⊙ up_output (ELEMENT-WISE MULTIPLICATION)',
  'activated_gate_component': 15.2421875,
  'up_output_component': -26.34375,
  'super_activation_result': -401.5,
  'formula': 'y[:, 1710] = SILU(W_gate[1710] @ x) * (W_up[1710] @ x)',
  

In [31]:
mathematical_analysis['weight_analysis']

{'target_channel_info': {'channel_index': 1710,
  'description': 'Channel 1710 in intermediate space where super activation occurs'},
 'gate_weights_analysis': {'operation': 'gate_output[1710] = W_gate[1710] @ x',
  'weight_vector_norm': 1.39453125,
  'max_weight': 0.317138671875,
  'min_weight': -0.2432861328125,
  'mean_weight': 0.0005812644958496094,
  'std_weight': 0.0308074951171875,
  'bias_term': None,
  'top_5_positive_weights': [{'dimension': 803, 'weight': 0.317138671875},
   {'dimension': 1542, 'weight': 0.266357421875},
   {'dimension': 1930, 'weight': 0.246337890625},
   {'dimension': 1344, 'weight': 0.2396240234375},
   {'dimension': 517, 'weight': 0.1917724609375}],
  'top_5_negative_weights': [{'dimension': 1981, 'weight': -0.2432861328125},
   {'dimension': 623, 'weight': -0.2041015625},
   {'dimension': 1916, 'weight': -0.149658203125},
   {'dimension': 967, 'weight': -0.1490478515625},
   {'dimension': 130, 'weight': -0.1474609375}]},
 'up_weights_analysis': {'operat